# Comparação de Algoritmos de Busca em Grafos — IA UFOP

**Problema:** Roteamento em Ouro Preto com função de custo (declividade, rugosidade, congestionamento).

**Algoritmos:** Dijkstra, A* e D* Lite.

**Cenários:** baseline, evento (ruas interditadas), clima (chuva — ladeiras com peso 200%).

In [1]:
import sys, os
sys.path.insert(0, '.')
from IPython.core.display import display_html
from src.graph import build_ouro_preto_example, build_ouro_preto_mariana_cachoeira
from src.algorithms import dijkstra, a_star, d_star_lite
from src.scenarios import apply_event_scenario, apply_climate_scenario, apply_climate_scenario_by_region, apply_congestion_scenario, apply_congestion_scenario_by_region, reset_scenarios
from src.metrics import measure_latency_ms

## 1. Grafo de exemplo (Ouro Preto)

Vértices: Praça Tiradentes, Terminal, Campus UFOP, Rua São José, Diogo de Vasconcelos, Rua do Pilar, Xavier da Veiga, Ladeira da Barra.
Arestas: distância, declividade (%), rugosidade (pé de moleque ~1.57), volume/capacidade.

In [2]:
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
G = build_ouro_preto_example(api_key)
START, GOAL = 'praca_tiradentes', 'campus'
print('Vértices:', list(G.nodes()))
print('Arestas (origem -> destino):', list(G.edges()))

Vértices: ['praca_tiradentes', 'terminal', 'campus', 'sao_jose', 'diogo_vasconcelos', 'rua_pilar', 'xavier_veiga', 'ladeira_barra', 'centro', 'antonio_dias', 'pilar', 'morro_santana', 'cabecas', 'lagoa', 'nossa_senhora_carmo', 'bauxita', 'vila_operaria', 'morro_cruzeiro', 'barra']
Arestas (origem -> destino): [('praca_tiradentes', 'terminal'), ('praca_tiradentes', 'campus'), ('praca_tiradentes', 'sao_jose'), ('praca_tiradentes', 'diogo_vasconcelos'), ('praca_tiradentes', 'rua_pilar'), ('praca_tiradentes', 'xavier_veiga'), ('praca_tiradentes', 'ladeira_barra'), ('praca_tiradentes', 'centro'), ('praca_tiradentes', 'antonio_dias'), ('praca_tiradentes', 'pilar'), ('praca_tiradentes', 'morro_santana'), ('praca_tiradentes', 'cabecas'), ('praca_tiradentes', 'lagoa'), ('praca_tiradentes', 'nossa_senhora_carmo'), ('praca_tiradentes', 'bauxita'), ('praca_tiradentes', 'vila_operaria'), ('praca_tiradentes', 'morro_cruzeiro'), ('praca_tiradentes', 'barra'), ('terminal', 'praca_tiradentes'), ('termi

In [3]:
from src.helpers.graph_helper import display_graph

# Gera grafo interativo (Dash Cytoscape) e exibe no notebook
display_graph(G, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

## 2. Cenário baseline — comparação dos três algoritmos

In [4]:
reset_scenarios(G)

lat_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Dijkstra:  caminho =', path_d, '| custo =', round(cost_d, 2))
print('A*:       caminho =', path_a, '| custo =', round(cost_a, 2))
print('D* Lite:  caminho =', path_dl, '| custo =', round(cost_dl, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d:.4f} ms | A*: {lat_a:.4f} ms | D* Lite: {lat_dl:.4f} ms')

Dijkstra:  caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
A*:       caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
D* Lite:  caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  Latência média (200 exec.): Dijkstra: 0.2100 ms | A*: 0.1245 ms | D* Lite: 0.1297 ms


## 3. Cenário de evento — ruas interditadas (Diogo de Vasconcelos)

O sistema deve sugerir rotas alternativas (Rua do Pilar ou Xavier da Veiga).

In [5]:
apply_event_scenario(G)

lat_d_e, (path_d_e, cost_d_e) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_e, (path_a_e, cost_a_e) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_e, (path_dl_e, cost_dl_e) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Com ruas interditadas:')
print('Dijkstra:  caminho =', path_d_e, '| custo =', round(cost_d_e, 2))
print('A*:       caminho =', path_a_e, '| custo =', round(cost_a_e, 2))
print('D* Lite:  caminho =', path_dl_e, '| custo =', round(cost_dl_e, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_e:.4f} ms | A*: {lat_a_e:.4f} ms | D* Lite: {lat_dl_e:.4f} ms')

Com ruas interditadas:
Dijkstra:  caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
A*:       caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
D* Lite:  caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  Latência média (200 exec.): Dijkstra: 0.2107 ms | A*: 0.1250 ms | D* Lite: 0.1334 ms


## 4. Cenário climático — chuva (ladeiras íngremes +200%)

In [6]:
reset_scenarios(G)
apply_climate_scenario(G, rain_multiplier=2.0)

lat_d_c, (path_d_c, cost_d_c) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_c, (path_a_c, cost_a_c) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_c, (path_dl_c, cost_dl_c) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Com chuva (ladeiras +200%):')
print('Dijkstra:  caminho =', path_d_c, '| custo =', round(cost_d_c, 2))
print('A*:       caminho =', path_a_c, '| custo =', round(cost_a_c, 2))
print('D* Lite:  caminho =', path_dl_c, '| custo =', round(cost_dl_c, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_c:.4f} ms | A*: {lat_a_c:.4f} ms | D* Lite: {lat_dl_c:.4f} ms')

Com chuva (ladeiras +200%):
Dijkstra:  caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
A*:       caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
D* Lite:  caminho = ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  Latência média (200 exec.): Dijkstra: 0.2034 ms | A*: 0.1250 ms | D* Lite: 0.1369 ms


### 4.1 Cenários combinados (Ouro Preto)

Bloqueio + chuva; bloqueio + chuva + alta densidade; chuva e congestionamento **por região** (alguns bairros com chuva/trânsito, outros não).

In [7]:
# Cenário A: bloqueio (Diogo de Vasconcelos) + chuva
reset_scenarios(G)
apply_event_scenario(G)
apply_climate_scenario(G, rain_multiplier=2.0)
display_graph(G, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

lat_d_ac, (path_d_ac, cost_d_ac) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_ac, (path_a_ac, cost_a_ac) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_ac, (path_dl_ac, cost_dl_ac) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('A — Bloqueio + Chuva:')
print('  Dijkstra:', path_d_ac, '| custo =', round(cost_d_ac, 2))
print('  A*:     ', path_a_ac, '| custo =', round(cost_a_ac, 2))
print('  D* Lite:', path_dl_ac, '| custo =', round(cost_dl_ac, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_ac:.4f} ms | A*: {lat_a_ac:.4f} ms | D* Lite: {lat_dl_ac:.4f} ms')

A — Bloqueio + Chuva:
  Dijkstra: ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  A*:      ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  D* Lite: ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  Latência média (200 exec.): Dijkstra: 0.2354 ms | A*: 0.1370 ms | D* Lite: 0.1331 ms


In [8]:
# Cenário B: bloqueio + chuva + alta densidade (congestionamento)
reset_scenarios(G)
apply_event_scenario(G)
apply_climate_scenario(G, rain_multiplier=2.0)
apply_congestion_scenario(G, congestion_factor=2.0)

lat_d_b, (path_d_b, cost_d_b) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_b, (path_a_b, cost_a_b) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_b, (path_dl_b, cost_dl_b) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('B — Bloqueio + Chuva + Congestionamento:')
print('  Dijkstra:', path_d_b, '| custo =', round(cost_d_b, 2))
print('  A*:     ', path_a_b, '| custo =', round(cost_a_b, 2))
print('  D* Lite:', path_dl_b, '| custo =', round(cost_dl_b, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_b:.4f} ms | A*: {lat_a_b:.4f} ms | D* Lite: {lat_dl_b:.4f} ms')

B — Bloqueio + Chuva + Congestionamento:
  Dijkstra: ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  A*:      ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  D* Lite: ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  Latência média (200 exec.): Dijkstra: 0.2131 ms | A*: 0.1284 ms | D* Lite: 0.1595 ms


In [9]:
# Cenário C: chuva e congestionamento por região (alguns bairros com chuva/trânsito, outros não)
reset_scenarios(G)
apply_climate_scenario_by_region(G, {"centro": 2.0, "praca_tiradentes": 2.0, "campus": 1.0})
apply_congestion_scenario_by_region(G, {"centro": 2.0, "sao_jose": 2.0})

lat_d_c2, (path_d_c2, cost_d_c2) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_c2, (path_a_c2, cost_a_c2) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_c2, (path_dl_c2, cost_dl_c2) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('C — Chuva + Congestionamento por região (centro/praça com chuva; centro/são_jose congestionados):')
print('  Dijkstra:', path_d_c2, '| custo =', round(cost_d_c2, 2))
print('  A*:     ', path_a_c2, '| custo =', round(cost_a_c2, 2))
print('  D* Lite:', path_dl_c2, '| custo =', round(cost_dl_c2, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_c2:.4f} ms | A*: {lat_a_c2:.4f} ms | D* Lite: {lat_dl_c2:.4f} ms')

C — Chuva + Congestionamento por região (centro/praça com chuva; centro/são_jose congestionados):
  Dijkstra: ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  A*:      ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  D* Lite: ['praca_tiradentes', 'pilar', 'campus'] | custo = 843.5
  Latência média (200 exec.): Dijkstra: 0.3795 ms | A*: 0.1385 ms | D* Lite: 0.1352 ms


## 5. Latência de re-roteamento (< 100 ms para tempo real)

In [10]:
reset_scenarios(G)
repetitions = 500

lat_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), repetitions)
lat_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL), repetitions)
lat_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), repetitions)

print(f'Latência média ({repetitions} execuções):')
print(f'  Dijkstra: {lat_d:.4f} ms')
print(f'  A*:       {lat_a:.4f} ms')
print(f'  D* Lite:  {lat_dl:.4f} ms')
print('  Meta: < 100 ms para uso em tempo real')

Latência média (500 execuções):
  Dijkstra: 0.2161 ms
  A*:       0.1274 ms
  D* Lite:  0.1319 ms
  Meta: < 100 ms para uso em tempo real


## 6. Tabela resumo para o relatório

In [11]:
import pandas as pd

reset_scenarios(G)
rows = []
for name, fn in [('Dijkstra', dijkstra), ('A*', a_star), ('D* Lite', d_star_lite)]:
    lat, (path, cost) = measure_latency_ms(lambda: fn(G, START, GOAL), 200)
    rows.append({'Algoritmo': name, 'Custo': round(cost, 2), 'Latência (ms)': round(lat, 4)})

df = pd.DataFrame(rows)
df

,Algoritmo,Custo,Latência (ms)
0,Dijkstra,843.5,0.2150
1,A*,843.5,0.1398
2,D* Lite,843.5,0.1419


## 7. Cenário regional: Ouro Preto + Mariana + Cachoeira do Campo

Grafo com nós nas três localidades. Origem/destino de exemplo: Ouro Preto (Praça Tiradentes) → Mariana (Centro).

In [12]:
G_regional = build_ouro_preto_mariana_cachoeira(api_key)
print('Vértices:', list(G_regional.nodes()))
print('Arestas (amostra):', list(G_regional.edges())[:15], '...')

Vértices: ['op_tiradentes', 'op_terminal', 'op_campus', 'op_centro', 'op_antonio_dias', 'op_pilar', 'op_morro_santana', 'op_cabecas', 'op_lagoa', 'op_nossa_senhora_carmo', 'op_bauxita', 'op_vila_operaria', 'op_morro_cruzeiro', 'op_barra', 'mariana_centro', 'mariana_praca', 'mariana_terminal', 'mariana_alvorada', 'mariana_rosario', 'mariana_santa_clara', 'mariana_santana', 'mariana_vila_rica', 'mariana_jardim_santana', 'mariana_liberdade', 'cachoeira_centro', 'cachoeira_igreja', 'cachoeira_praca_coronel', 'cachoeira_dom_bosco', 'cachoeira_praca_dom_bosco']
Arestas (amostra): [('op_tiradentes', 'op_terminal'), ('op_tiradentes', 'op_campus'), ('op_tiradentes', 'op_centro'), ('op_tiradentes', 'op_antonio_dias'), ('op_tiradentes', 'op_pilar'), ('op_tiradentes', 'op_morro_santana'), ('op_tiradentes', 'op_cabecas'), ('op_tiradentes', 'op_lagoa'), ('op_tiradentes', 'op_nossa_senhora_carmo'), ('op_tiradentes', 'op_bauxita'), ('op_tiradentes', 'op_vila_operaria'), ('op_tiradentes', 'op_morro_cru

In [13]:

START_REGIONAL, GOAL_REGIONAL = 'cachoeira_praca_dom_bosco', 'mariana_jardim_santana'

In [14]:
from src.helpers.graph_helper import spread_positions

pos_r = {n: G_regional.nodes[n]["pos"] for n in G_regional.nodes()}
pos_r = spread_positions(pos_r, min_distance=500, iterations=8, factor=0.4)
def _short(s):
    s = s.replace("_", " ").title()
    return s[:18] + "…" if len(s) > 18 else s
labels_r = {n: _short(n) for n in G_regional.nodes()}

fig, ax = plt.subplots(figsize=(14, 10))
nx.draw_networkx_nodes(G_regional, pos_r, node_color="#87CEEB", node_size=500, ax=ax, edgecolors="#333", linewidths=1.5)
nx.draw_networkx_edges(G_regional, pos_r, edge_color="#888", alpha=0.4, arrows=True, arrowsize=10, ax=ax, connectionstyle="arc3,rad=0.06")
nx.draw_networkx_labels(G_regional, pos_r, labels=labels_r, font_size=8, ax=ax, font_weight="bold")
ax.set_title("Grafo regional — Ouro Preto + Mariana + Cachoeira do Campo (posições reais)", fontsize=14)
ax.set_aspect("equal")
ax.axis("off")
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

### 7.1 Baseline regional (op_tiradentes → mariana_centro)

In [ ]:
reset_scenarios(G_regional)

lat_d_r, (path_d_r, cost_d_r) = measure_latency_ms(lambda: dijkstra(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_a_r, (path_a_r, cost_a_r) = measure_latency_ms(lambda: a_star(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_dl_r, (path_dl_r, cost_dl_r) = measure_latency_ms(lambda: d_star_lite(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)

print('Baseline regional:')
print('Dijkstra:  caminho =', path_d_r, '| custo =', round(cost_d_r, 2))
print('A*:       caminho =', path_a_r, '| custo =', round(cost_a_r, 2))
print('D* Lite:  caminho =', path_dl_r, '| custo =', round(cost_dl_r, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_r:.4f} ms | A*: {lat_a_r:.4f} ms | D* Lite: {lat_dl_r:.4f} ms')

### 7.2 Cenário de evento — bloqueio de trecho (regional)

Exemplo: interditar aresta op_tiradentes → op_centro; o sistema deve sugerir rota alternativa.

In [ ]:
apply_event_scenario(G_regional, blocked_edges=[('op_tiradentes', 'op_centro'), ('op_centro', 'op_tiradentes')])

lat_d_re, (path_d_re, cost_d_re) = measure_latency_ms(lambda: dijkstra(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_a_re, (path_a_re, cost_a_re) = measure_latency_ms(lambda: a_star(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_dl_re, (path_dl_re, cost_dl_re) = measure_latency_ms(lambda: d_star_lite(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)

print('Com bloqueio (op_tiradentes ↔ op_centro):')
print('Dijkstra:  caminho =', path_d_re, '| custo =', round(cost_d_re, 2))
print('A*:       caminho =', path_a_re, '| custo =', round(cost_a_re, 2))
print('D* Lite:  caminho =', path_dl_re, '| custo =', round(cost_dl_re, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_re:.4f} ms | A*: {lat_a_re:.4f} ms | D* Lite: {lat_dl_re:.4f} ms')

### 7.3 Cenário climático — chuva (regional)

In [ ]:
reset_scenarios(G_regional)
apply_climate_scenario(G_regional, rain_multiplier=2.0)

lat_d_rc, (path_d_rc, cost_d_rc) = measure_latency_ms(lambda: dijkstra(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_a_rc, (path_a_rc, cost_a_rc) = measure_latency_ms(lambda: a_star(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_dl_rc, (path_dl_rc, cost_dl_rc) = measure_latency_ms(lambda: d_star_lite(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)

print('Com chuva (ladeiras +200%):')
print('Dijkstra:  caminho =', path_d_rc, '| custo =', round(cost_d_rc, 2))
print('A*:       caminho =', path_a_rc, '| custo =', round(cost_a_rc, 2))
print('D* Lite:  caminho =', path_dl_rc, '| custo =', round(cost_dl_rc, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_rc:.4f} ms | A*: {lat_a_rc:.4f} ms | D* Lite: {lat_dl_rc:.4f} ms')

### 7.3.1 Cenários combinados (regional)

Bloqueio + chuva; bloqueio + chuva + congestionamento; chuva e congestionamento **por região** (OP com chuva/trânsito, Mariana/Cachoeira normais ou leves).

In [ ]:
# Regional A: bloqueio (op_tiradentes ↔ op_centro) + chuva
reset_scenarios(G_regional)
apply_event_scenario(G_regional, blocked_edges=[('op_tiradentes', 'op_centro'), ('op_centro', 'op_tiradentes')])
apply_climate_scenario(G_regional, rain_multiplier=2.0)
lat_d_ra, (path_d_ra, cost_d_ra) = measure_latency_ms(lambda: dijkstra(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_a_ra, (path_a_ra, cost_a_ra) = measure_latency_ms(lambda: a_star(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_dl_ra, (path_dl_ra, cost_dl_ra) = measure_latency_ms(lambda: d_star_lite(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
print('A — Bloqueio + Chuva (regional):')
print('  Dijkstra:', path_d_ra, '| custo =', round(cost_d_ra, 2))
print('  A*:     ', path_a_ra, '| custo =', round(cost_a_ra, 2))
print('  D* Lite:', path_dl_ra, '| custo =', round(cost_dl_ra, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_ra:.4f} ms | A*: {lat_a_ra:.4f} ms | D* Lite: {lat_dl_ra:.4f} ms')

In [ ]:
# Regional B: bloqueio + chuva + congestionamento
reset_scenarios(G_regional)
apply_event_scenario(G_regional, blocked_edges=[('op_tiradentes', 'op_centro'), ('op_centro', 'op_tiradentes')])
apply_climate_scenario(G_regional, rain_multiplier=2.0)
apply_congestion_scenario(G_regional, congestion_factor=2.0)
lat_d_rb, (path_d_rb, cost_d_rb) = measure_latency_ms(lambda: dijkstra(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_a_rb, (path_a_rb, cost_a_rb) = measure_latency_ms(lambda: a_star(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_dl_rb, (path_dl_rb, cost_dl_rb) = measure_latency_ms(lambda: d_star_lite(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
print('B — Bloqueio + Chuva + Congestionamento (regional):')
print('  Dijkstra:', path_d_rb, '| custo =', round(cost_d_rb, 2))
print('  A*:     ', path_a_rb, '| custo =', round(cost_a_rb, 2))
print('  D* Lite:', path_dl_rb, '| custo =', round(cost_dl_rb, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_rb:.4f} ms | A*: {lat_a_rb:.4f} ms | D* Lite: {lat_dl_rb:.4f} ms')

In [ ]:
# Regional C: chuva e congestionamento por região (OP com chuva e trânsito; Mariana/Cachoeira leves)
reset_scenarios(G_regional)
apply_climate_scenario_by_region(G_regional, {"op": 2.0, "mariana": 1.0, "cachoeira": 1.5})
apply_congestion_scenario_by_region(G_regional, {"op": 2.0, "mariana": 1.0, "cachoeira": 1.0})
lat_d_rc2, (path_d_rc2, cost_d_rc2) = measure_latency_ms(lambda: dijkstra(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_a_rc2, (path_a_rc2, cost_a_rc2) = measure_latency_ms(lambda: a_star(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
lat_dl_rc2, (path_dl_rc2, cost_dl_rc2) = measure_latency_ms(lambda: d_star_lite(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
print('C — Chuva + Congestionamento por região (OP: chuva 2x + congestionamento 2x; Mariana/Cachoeira: normal):')
print('  Dijkstra:', path_d_rc2, '| custo =', round(cost_d_rc2, 2))
print('  A*:     ', path_a_rc2, '| custo =', round(cost_a_rc2, 2))
print('  D* Lite:', path_dl_rc2, '| custo =', round(cost_dl_rc2, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_rc2:.4f} ms | A*: {lat_a_rc2:.4f} ms | D* Lite: {lat_dl_rc2:.4f} ms')

### 7.4 Latência de re-roteamento (regional)

In [ ]:
reset_scenarios(G_regional)
repetitions_r = 300

lat_d_r, (path_d_r, cost_d_r) = measure_latency_ms(lambda: dijkstra(G_regional, START_REGIONAL, GOAL_REGIONAL), repetitions_r)
lat_a_r, (path_a_r, cost_a_r) = measure_latency_ms(lambda: a_star(G_regional, START_REGIONAL, GOAL_REGIONAL), repetitions_r)
lat_dl_r, (path_dl_r, cost_dl_r) = measure_latency_ms(lambda: d_star_lite(G_regional, START_REGIONAL, GOAL_REGIONAL), repetitions_r)

print(f'Latência média ({repetitions_r} execuções) — cenário regional:')
print(f'  Dijkstra: {lat_d_r:.4f} ms')
print(f'  A*:       {lat_a_r:.4f} ms')
print(f'  D* Lite:  {lat_dl_r:.4f} ms')

### 7.5 Tabela resumo — cenário regional

In [ ]:
reset_scenarios(G_regional)
rows_r = []
for name, fn in [('Dijkstra', dijkstra), ('A*', a_star), ('D* Lite', d_star_lite)]:
    lat, (path, cost) = measure_latency_ms(lambda: fn(G_regional, START_REGIONAL, GOAL_REGIONAL), 200)
    rows_r.append({'Algoritmo': name, 'Custo': round(cost, 2), 'Latência (ms)': round(lat, 4)})

df_regional = pd.DataFrame(rows_r)
df_regional